In [1]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pickle
import random

from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization

In [2]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        if K.image_data_format() == 'channels_first':
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Conv2D(32, (3, 3), padding='same', use_bias=False, activation='relu', input_shape=inputShape))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.35))
        
        model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.35))
        
        model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.35))
        
        model.add(Conv2D(256, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(256, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.35))
        
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(classes, activation='softmax'))
        
        return model

In [3]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (168, 168, 3)

data = []
labels = []

print('[INFO] loading images...')
imagePaths = sorted(list(paths.list_images('dataset')))
random.seed(2020)
random.shuffle(imagePaths)

[INFO] loading images...


In [4]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [5]:
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)
print('[INFO] data matrix: {:.2f}MB'.format(data.nbytes / (1024 * 1000.0)))

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

[INFO] data matrix: 2009.64MB


In [6]:
aug = ImageDataGenerator(rotation_range=40,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [7]:
print('[INFO] compiling model...')
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()

[INFO] compiling model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 168, 168, 32)      864       
_________________________________________________________________
batch_normalization (BatchNo (None, 168, 168, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        18432     
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (

In [8]:
##### print('[INFO] training network...')
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS,
    verbose=1
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 75 steps, validate on 608 samples
Epoch 1/100
75/75 [==============================] - 155s 2s/step - loss: 2.5199 - accuracy: 0.3048 - val_loss: 4.3242 - val_accuracy: 0.1513
Epoch 2/100
75/75 [==============================] - 11s 143ms/step - loss: 2.0250 - accuracy: 0.4062 - val_loss: 4.6585 - val_accuracy: 0.1266
Epoch 3/100
75/75 [==============================] - 11s 144ms/step - loss: 1.6371 - accuracy: 0.4925 - val_loss: 2.6960 - val_accuracy: 0.2533
Epoch 4/100
75/75 [==============================] - 11s 143ms/step - loss: 1.5458 - accuracy: 0.5046 - val_loss: 1.9976 - val_accuracy: 0.3602
Epoch 5/100
75/75 [==============================] - 11s 142ms/step - loss: 1.4439 - accuracy: 0.5434 - val_loss: 1.6175 - val_accuracy: 0.4556
Epoch 6/100
75/75 [==============================] - 11s 140ms/step - loss: 1.3168 - accuracy: 0.5709 - val_loss: 1.6518 - val_accuracy: 

Epoch 55/100
75/75 [==============================] - 11s 142ms/step - loss: 0.2945 - accuracy: 0.8987 - val_loss: 0.9133 - val_accuracy: 0.7944
Epoch 56/100
75/75 [==============================] - 11s 143ms/step - loss: 0.2692 - accuracy: 0.9087 - val_loss: 0.8760 - val_accuracy: 0.7730
Epoch 57/100
75/75 [==============================] - 11s 143ms/step - loss: 0.2616 - accuracy: 0.9174 - val_loss: 1.0275 - val_accuracy: 0.7549
Epoch 58/100
75/75 [==============================] - 11s 143ms/step - loss: 0.2324 - accuracy: 0.9270 - val_loss: 1.2893 - val_accuracy: 0.7549
Epoch 59/100
75/75 [==============================] - 11s 142ms/step - loss: 0.2769 - accuracy: 0.9078 - val_loss: 1.2500 - val_accuracy: 0.7467
Epoch 60/100
75/75 [==============================] - 11s 142ms/step - loss: 0.3052 - accuracy: 0.8970 - val_loss: 1.6989 - val_accuracy: 0.7204
Epoch 61/100
75/75 [==============================] - 11s 142ms/step - loss: 0.2440 - accuracy: 0.9128 - val_loss: 0.8522 - val_ac

In [9]:
model.save('model.h5')

f = open('label.pickle', 'wb')
f.write(pickle.dumps(lb))
f.close()

In [10]:
plt.style.use('ggplot')
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history['loss'], label='train_loss')
plt.plot(np.arange(0, N), H.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, N), H.history['accuracy'], label='train_accuracy')
plt.plot(np.arange(0, N), H.history['val_accuracy'], label='val_accuracy')
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend(loc='upper left')
plt.savefig('plot.png')